In [1]:
import lib.facebook_scraper as fs
import lib.utils as utils
import pandas as pd
import numpy as np
import os
import time

In [2]:
post_ids_df = pd.read_csv('data/vebay69/clean/post_ids.csv')
post_ids_df.head()

,post_id
0,685638947004646
1,685577330344141
2,685111467057394
3,684686633766544
4,684603707108170


In [3]:
# Lấy danh sách post_id từ vị trí index của post_ids_df: pd.DataFrame với số lượng là number
def get_list_post_id(index: int, number: int) -> list:
    global post_ids_df
    if index < 0:
        index = 0
    if index > post_ids_df.shape[0]:
        return []
    if index + number > post_ids_df.shape[0]:
        return post_ids_df.iloc[index:]['post_id'].to_list()
    return post_ids_df.iloc[index:index+number]['post_id'].to_list()

In [4]:
def extract_comment(comments, post_id):
    cnt_comment = 0
    cnt_reply = 0
    global comment_list
    for comment in comments:
        comment['post_id'] = post_id
        comment_list.append(comment)
        cnt_comment += 1
        time.sleep(np.random.uniform(0, 0.5))
        for reply in comment['replies']:
            reply['post_id'] = post_id
            comment_list.append(reply)
            cnt_reply += 1
            time.sleep(np.random.uniform(0, 0.5))

    print(f'\tExtracted {cnt_comment} comments and {cnt_reply} replies')
    return cnt_comment + cnt_reply

In [5]:
index_post_id_file_path = os.path.join('data','vebay69','url','i_post_id_comment.txt')

### Rerun with change cookie

In [6]:
fs.set_cookies('./cookies/cookie_1.txt')
# fs.enable_logging()

### Rerun

In [7]:
NUMBER_POST = 30
post_list = []
comment_list = []
total_comment_real = 0

In [8]:
index_post_id = utils.read_index_file(file_path=index_post_id_file_path)
print(index_post_id)
posts_ids = get_list_post_id(index_post_id, NUMBER_POST)
posts_ids

152


[648745990693942,
 648129900755551,
 648136317421576,
 646648884236986,
 647745007460707,
 647707500797791,
 647178930850648,
 646704834231391,
 646610584240816,
 646578170910724,
 646251794276695,
 646239270944614,
 646182897616918,
 646120450956496,
 645592771009264,
 645129864388888,
 645101634391711,
 644774441091097,
 644783744423500,
 644728431095698,
 644644097770798,
 644200194481855,
 644177814484093,
 643836757851532,
 643680344533840,
 643285894573285,
 643214227913785,
 642701377965070,
 642353657999842,
 642223291346212]

In [9]:
try:
    for post in fs.get_posts(
        post_urls=posts_ids,
        options={
            "comments": True,
            "allow_extra_requests": True,
            "comments": "generator",
        },
        timeout=120,
    ):
        print(post)
        num_comment = extract_comment(post['comments_full'], post['original_request_url'])
        post_list.append(post)
        print(f'\tTotal {num_comment}/{post["comments"]} comments')
        total_comment_real += post['comments']
        utils.sleep(np.random.randint(30, 60))
except fs.exceptions.TemporarilyBanned:
    print("TEMPORARILY BANNED")
except fs.exceptions.AccountDisabled:
    print("ACCOUNT DISABLED")
except Exception as e:
    print(e)

/workspaces/analyze-facebook-page/lib/facebook_scraper/facebook_scraper.py:912: UserWarning: Facebook says 'Unsupported Browser'
  warnings.warn(f"Facebook says 'Unsupported Browser'")


{'original_request_url': 648745990693942, 'post_url': 'https://facebook.com/story.php?story_fbid=648745990693942&id=100066756416622', 'post_id': 648745990693942, 'text': 'Hồi máo\n#Panda', 'post_text': 'Hồi máo\n#Panda', 'shared_text': '', 'original_text': None, 'time': datetime.datetime(2023, 9, 12, 4, 36, 25), 'timestamp': 1694493385, 'image': 'https://scontent-sin6-4.xx.fbcdn.net/v/t39.30808-6/377940403_648745680693973_7063282598504420586_n.jpg?stp=cp0_dst-jpg_e15_fr_q65&_nc_cat=110&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=I-5LMIre2PMAX8Qp9Jo&_nc_ht=scontent-sin6-4.xx&oh=00_AfA_Sap8vPqugwXSU4PKiDR_G2UFmMfYOWL649mieQfp_g&oe=656A1915&manual_redirect=1', 'image_lowquality': 'https://scontent-sin6-4.xx.fbcdn.net/v/t39.30808-6/377940403_648745680693973_7063282598504420586_n.jpg?stp=cp0_dst-jpg_e15_p320x320_q65&_nc_cat=110&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=I-5LMIre2PMAX8Qp9Jo&_nc_ht=scontent-sin6-4.xx&oh=00_AfACM0sCdnykhIBNg58z95kE_cVXB94R5yW3P7v7G1Pi7A&oe=656A1915', 

In [10]:
print(NUMBER_POST)
print(f"Số posts đã crawl: {len(post_list)}")
print(f"Số comments đã crawl: {len(comment_list)}/{total_comment_real}")

30
Số posts đã crawl: 13
Số comments đã crawl: 1758/2193


In [11]:
if post_list:
    utils.save_data(
        data_list=comment_list,
        folder_path=os.path.join('data','vebay69','raw'),
        type="comments"
    )
utils.write_index_file(
    file_path=index_post_id_file_path,
    index=index_post_id + len(post_list)
)

Save comments data: ./data/vebay69/raw/comments_2023-11-28_14-28-20.csv
Save index 165 to resume: ./data/vebay69/url/i_post_id_comment.txt
